Create_contact_info

In [1]:
import os
import numpy as np
import h5py
import glob
import argparse
import sys

# ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(ROOT_DIR)

from contact_graspnet.data import PointCloudReader
from contact_graspnet.mesh_utils import in_collision_with_gripper, grasp_contact_location

import pprint as pp 
from pathlib import Path 

from contact_graspnet.mesh_utils import create_gripper
import trimesh 


The notebook takes as input a directory of the format 

```
mobile_pnp/src/ArmGraspObjectClutterReconstruction-v0_antipodal_grasp_generation/
└───{env_name}_eval_results
│   └───{obj_name}
│       │   grasp_pose_and_labels.npy
```

and outputs a directory of the format 

```
reconstructed/mesh_contacts/
└───{env_name}
│       │   {obj_name}.npz
```

where each {obj_name}.npz file contains the contact points and other information information related to the grasps for the object in the scene {env_name}

In [2]:
from utils import load_label_from_bfs, get_path_to_recon_mesh

mobile_pnp_dir = Path('/home/robin/Projects/git_manage/mobile_pnp')

num_test_scene = 30 

for env_idx in range(num_test_scene):

    recon_env_name = f'ArmGraspObjectClutterReconstruction_{env_idx}-v0'

    pcreader = PointCloudReader(root_folder=None)

    obj_name_to_bfs_label = load_label_from_bfs(
        recon_env_name, mobile_pnp_dir
    )

    list_obj_name = obj_name_to_bfs_label.keys()#os.listdir(labels_for_env)

    for obj_name in list_obj_name:

        art_info = obj_name_to_bfs_label[obj_name]

        if art_info == None: # handle objects with no grasps
            continue

        Tarr_world_cgngripper = art_info['Tarr_world_cgngripper']
        grasp_success_labels = art_info['grasp_success_labels']

        env_name = art_info['env_name']
        target_obj_name = art_info['target_obj_name']

        T_world_obj = art_info['T_world_obj']

        # Begin function read_object_grasp_data_acronym
        mesh_fname = get_path_to_recon_mesh(
            env_name, target_obj_name, mobile_pnp_dir)
        mesh_scale = 1.     

        # How to interpret the frame these grasps are represented wrt
        # https://github.com/NVlabs/acronym/blob/main/scripts/acronym_visualize_grasps.py
        # (2000, 4, 4) 
        grasps = Tarr_world_cgngripper
        T_obj_world = np.linalg.inv(T_world_obj)
        grasps = T_obj_world @ Tarr_world_cgngripper

        # (2000,)
        success = grasp_success_labels

        positive_grasps = grasps[success==1, :, :]
        negative_grasps = grasps[success==0, :, :]

        # print('positive grasps: {} negative grasps: {}'.format(positive_grasps.shape, negative_grasps.shape))

        output_grasps, output_labels, cad_path, cad_scale = grasps, success, mesh_fname, mesh_scale
        # End # Begin function read_object_grasp_data_acronym

        # This fn will also shift the mesh vertices
        # such that the mean of the mesh is at origin
        pcreader._renderer._load_object(cad_path, cad_scale)

        context = pcreader._renderer._cache[(cad_path,cad_scale)]
        obj_mesh = context['tmesh']
        obj_mesh_mean = context['mesh_mean']

        # print('obj_mesh_mean', obj_mesh_mean)

        output_grasps[:,:3,3] -= obj_mesh_mean

        # Begin grasps_contact_info(output_grasps, list(output_labels), obj_mesh, check_collisions=False)
        grasp_tfs, successfuls, obj_mesh = output_grasps, list(output_labels), obj_mesh

        # Begin grasp_contact_location
        transforms, successfuls = grasp_tfs, successfuls

        collisions = [0]*len(successfuls)   
        object_mesh = obj_mesh
        gripper_name = 'panda' 
        silent = False

        res = []
        gripper = create_gripper(gripper_name)

        intersector = trimesh.ray.ray_triangle.RayMeshIntersector(object_mesh)

        for p, colliding, outcome in zip(transforms, collisions, successfuls):

            contact_dict = {}
            contact_dict['collisions'] = 0
            contact_dict['valid_locations'] = 0
            contact_dict['successful'] = outcome
            contact_dict['grasp_transform'] = p
            contact_dict['contact_points'] = []
            contact_dict['contact_directions'] = []
            contact_dict['contact_face_normals'] = []
            contact_dict['contact_offsets'] = []

            assert not colliding

            ray_origins, ray_directions = gripper.get_closing_rays_contact(p)

            locations, index_ray, index_tri = intersector.intersects_location(
                ray_origins, ray_directions, multiple_hits=False)

            if len(locations) > 0:
                # this depends on the width of the gripper
                # [ True  True]
                valid_locations = np.linalg.norm(ray_origins[index_ray]-locations, axis=1) <= 2.0*gripper.q

                if sum(valid_locations) > 1:
                    contact_dict['valid_locations'] = 1
                    contact_dict['contact_points'] = locations[valid_locations]
                    contact_dict['contact_face_normals'] = object_mesh.face_normals[index_tri[valid_locations]]
                    contact_dict['contact_directions'] = ray_directions[index_ray[valid_locations]]
                    contact_dict['contact_offsets'] = np.linalg.norm(ray_origins[index_ray[valid_locations]] - locations[valid_locations], axis=1)
                    
                    res.append(contact_dict)

        contact_dicts = res 

        contact_dict_of_arrays = {}
        for d in contact_dicts:
            for k in d:
                contact_dict_of_arrays.setdefault(k,[]).append(d[k])

        # This saves it to 
        # reconstructed/mesh_contacts/{env_name}/{target_obj_name}.npz
        contact_dir_path = Path(f'reconstructed/mesh_contacts/{env_name}')
        contact_dir_path.mkdir(exist_ok=True, parents=True)
        contact_dir_path /= target_obj_name

        np.savez(contact_dir_path, **contact_dict_of_arrays)
    

********** terminating renderer **************
********** terminating renderer **************
********** terminating renderer **************
/home/robin/Projects/git_manage/mobile_pnp/src/ArmGraspObjectClutterReconstruction-v0_antipodal_grasp_generation/ArmGraspObjectClutterReconstruction-v0_Ditto_grasp_labels_202206151746/ArmGraspObjectClutterReconstruction_3-v0_eval_results/bottle_2/grasp_pose_and_labels.npy is not found. The grasps and labels for this object                 is treated as not found, i.e. zero grasp is found. Please verify.
********** terminating renderer **************
********** terminating renderer **************
********** terminating renderer **************
********** terminating renderer **************
/home/robin/Projects/git_manage/mobile_pnp/src/ArmGraspObjectClutterReconstruction-v0_antipodal_grasp_generation/ArmGraspObjectClutterReconstruction-v0_Ditto_grasp_labels_202206151746/ArmGraspObjectClutterReconstruction_7-v0_eval_results/bottle_0/grasp_pose_and_la

In [8]:
contact_dict_of_arrays['contact_points'][0].shape

(2, 3)

In [3]:
# Viz for debugging

from acronym_tools.acronym import create_gripper_marker

sg = [create_gripper_marker().apply_transform(t) for t in grasps]

trimesh.Scene(object_mesh + sg).show()
